In [1]:
import os
import logging
import logging.config
import threading
import queue
import time

In [2]:
logging.config.fileConfig('../logging.conf')

# create logger
logger = logging.getLogger('main')

In [3]:
logger.debug('This is a debug message')
logger.info('This is an info message')
logger.warning('This is a warning message')
logger.error('This is an error message')
logger.critical('This is a critical message')

03-02 10:36:43 main   | This is a debug message
03-02 10:36:43 main   | This is an info message
03-02 10:36:43 main   | This is a warning message
03-02 10:36:43 main   | This is an error message
03-02 10:36:43 main   | This is a critical message


In [4]:
def print_numbers():
    logger = logging.getLogger('numbers')
    for i in range(1, 4):
        logger.info(f"Number: {i}")
        time.sleep(1)

def print_letters():
    logger = logging.getLogger('letters')
    for letter in ['A', 'B', 'C']:
        logger.info(f"Letter: {letter}")
        time.sleep(1.5)

# Create threads
thread1 = threading.Thread(target=print_numbers)
thread2 = threading.Thread(target=print_letters)

# Start threads
thread1.start()
thread2.start()

# Wait for both threads to complete
thread1.join()
thread2.join()

print("Both threads finished execution.")

03-02 10:36:43 numbers | Number: 1
03-02 10:36:43 letters | Letter: A
03-02 10:36:44 numbers | Number: 2
03-02 10:36:44 letters | Letter: B
03-02 10:36:45 numbers | Number: 3
03-02 10:36:46 letters | Letter: C
Both threads finished execution.


In [5]:
# Shared variable
counter = 0

# Create a queue for thread communication
command_queue = queue.Queue()

def increment_counter():
    """Thread function to signal incrementing the counter."""
    for _ in range(3):
        # Instead of modifying the counter directly, we put an 'increment' command into the queue
        command_queue.put('increment')
        time.sleep(1.0)

# Thread target functions
def thread_function(name):
    logger = logging.getLogger(f"thread {name}")
    logger.info(f"Thread {name}: starting")
    increment_counter()
    logger.info(f"Thread {name}: finishing")

# Main thread operation to update the shared variable based on commands from the queue
def process_queue_commands():
    global counter
    logger = logging.getLogger()
    while True:
        command = command_queue.get()
        if command == 'increment':
            counter += 1
            logger.info(f"Updated counter to {counter}")
        command_queue.task_done()
        if counter >= 6:  # Assuming we want to stop after 20 increments
            break

# Create two threads
thread1 = threading.Thread(target=thread_function, args=(1,))
thread2 = threading.Thread(target=thread_function, args=(2,))

# Start threads
thread1.start()
thread2.start()

# Start processing commands from the queue
process_queue_commands()

# Wait for all items in the queue to be processed
command_queue.join()

# Wait for threads to complete
thread1.join()
thread2.join()

print("All threads finished execution. Final counter value:", counter)


03-02 10:36:47 thread 1 | Thread 1: starting
03-02 10:36:47 thread 2 | Thread 2: starting
03-02 10:36:47 root   | Updated counter to 1
03-02 10:36:47 root   | Updated counter to 2
03-02 10:36:48 root   | Updated counter to 3
03-02 10:36:48 root   | Updated counter to 4
03-02 10:36:49 root   | Updated counter to 5
03-02 10:36:49 root   | Updated counter to 6
03-02 10:36:50 thread 2 | Thread 2: finishing
03-02 10:36:50 thread 1 | Thread 1: finishing
All threads finished execution. Final counter value: 6
